In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
news=pd.read_csv("preprocessed_news.csv")
user = pd.read_csv('behaviors.tsv', sep = '\t', names=["impressionId","userId","timestamp","click_history","impressions"])
newsEmbedding=np.load('news_embeddings_numpy.npy')

In [3]:
user = user.drop_duplicates(["userId"])
user = user.dropna()

In [6]:
def process_impression(impression_list):
    list_of_strings = impression_list.split()
    click = [x.split('-')[0] for x in list_of_strings if x.split('-')[1] == '1']
    non_click = [x.split('-')[0] for x in list_of_strings if x.split('-')[1] == '0']
    return click,non_click

user['click'], user['noclicks'] = zip(*user['impressions'].map(process_impression))

In [8]:
news_dict = {}
news_ids = np.array(news['itemId'])

In [9]:
for i in range(len(news_ids)):
    news_dict[news_ids[i]] = newsEmbedding[i]

In [11]:
click_history = np.array(user['click_history'])

In [12]:
type(click_history[0])
click_history1 = []

In [14]:
for i in range(len(click_history)):
    # click_history[i] = click_history[i].split(" ")[0]
    click_history1.append(click_history[i].split(" "))


In [16]:
user_dict ={}
user_id = np.array(user['userId'])

In [17]:
for i in range(len(click_history)):
    user_dict[user_id[i]] = click_history1[i]

In [18]:
k = []
user_emb = []
# weights = []
for i in user_dict.keys():
    empty=np.zeros(384)
    for j in user_dict[i]:
        empty += news_dict[j]
    empty /= len(user_dict[i])
    k.append(empty)

User Representation - 2

In [21]:

# Let ‘k’ be defined as the average news embedding of the click history. Let us consider the weights to be the the dot product of k with the news
# embeddings of the click history. Then we take the weighted average of the embeddings of the click history with the above formed weights to form the user embedding.


weights = []
i_count = 0
for i in user_dict.keys():
    sum = 0
    for j in user_dict[i]:
        weights.append(np.dot(k[i_count], news_dict[j].T))
    i_count += 1

In [22]:
weights = np.array(weights)
sum = np.sum(weights)

In [24]:
user_emb = []
i_count = 0
for i in user_dict.keys():
    partial_sum = 0
    empty=np.zeros(384)
    for j in user_dict[i]:
        empty += (weights[i_count]*news_dict[j])
        partial_sum += weights[i_count]
        i_count += 1
    user_emb.append(empty/partial_sum)
    

In [25]:
np.save("new_user_embiddings.npy", user_emb)

User Representation - 3

In [27]:
# We formed n (number of news articles in the click history) vectors, by taking the weighted average, with weights, being the dot product of  Vi and Vj. The we take average 
# of these vectora to form the final user representation.

user_emb_2 = []
for i in user_dict.keys():
    v_dash = []
    weights = []
    middle = np.zeros(384)
    i_count = 0
    for j in user_dict[i]:
        for p in user_dict[i]:
            weights.append(np.dot(news_dict[j], news_dict[p].T))


    for j in user_dict[i]:
        empty=np.zeros(384)
        partial_sum = 0
        for k in user_dict[i]:
            empty += (weights[i_count]*news_dict[k])
            partial_sum += weights[i_count]
            i_count += 1
        
        v_dash.append(empty/partial_sum)
    # break
    for a in range(len(v_dash)):
        middle += v_dash[a]
        middle = middle/len(v_dash)
    user_emb_2.append(middle)


In [141]:
np.save("new_user_embiddings_2.npy", user_emb_2)